In [2]:
import requests
import time
import pandas as pd

TOKEN = "d5c3d78fc111d88a0a37b4ab8f83cbd5"
url = "https://ows.goszakup.gov.kz/v3/subject/all"
parquet_file = "goszakup_subjAll_data.parquet"

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

params = {"limit": 500}  # Загружаем 500 записей за раз
all_data = []  
search_after = None  

while len(all_data)<1500:
    try:
        if search_after:  
            params.update({"page": "next", "search_after": search_after})

        response = requests.get(url, headers=headers, params=params, timeout=30)
        response.raise_for_status()
        data = response.json()

        items = data.get("items", [])
        if not items:
            print("Загрузка завершена, новых данных нет.")
            break

        all_data.extend(items)
        print(f"Загружено {len(all_data)} записей...")

        search_after = items[-1]["pid"]

        # Промежуточное сохранение каждые 5000 записей
        if len(all_data) % 1000 < 500:
            pd.DataFrame(all_data).to_parquet(parquet_file, index=False, engine="pyarrow")
            print("Промежуточное сохранение...")

        time.sleep(5)

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
        print("Ждём 30 сек и пробуем снова...")
        time.sleep(30)

# Финальное сохранение
df = pd.DataFrame(all_data)
df.to_parquet(parquet_file, index=False, engine="pyarrow")

print(f"Данные сохранены в {parquet_file}")

Загружено 500 записей...
Загружено 1000 записей...
Промежуточное сохранение...
Ошибка запроса: HTTPSConnectionPool(host='ows.goszakup.gov.kz', port=443): Read timed out.
Ждём 30 сек и пробуем снова...
Загружено 1500 записей...
Данные сохранены в goszakup_subjAll_data.parquet


In [3]:
df = pd.read_parquet("goszakup_subjAll_data.parquet")
df.head()

,pid,bin,iin,inn,unp,regdate,crdate,number_reg,series,name_ru,...,mark_world_company,mark_state_monopoly,mark_natural_monopoly,mark_patronymic_producer,mark_patronymic_supplyer,mark_small_employer,type_supplier,is_single_org,system_id,index_date
0,771933,,870802350452,,,None,2025,,,"""Crossover Production""",...,0,0,0,0,0,0,2,0,3,2025-03-18 16:40:02
1,771932,,581217302335,,,None,2025,,,КИМ БОРИС,...,0,0,0,0,0,0,2,0,3,2025-03-18 16:40:02
2,771930,,870729301273,,,None,2025,,,ИП НБК,...,0,0,0,0,0,0,3,0,3,2025-03-18 16:40:02
3,771929,211240000776,,,,2022-02-11 00:00:00,2025,21949,,"ТОО ""Отан Групп ЛТД""",...,0,0,0,0,0,0,1,0,3,2025-03-18 16:40:02
4,771926,,840409401270,,,None,2025,,,НҰР-ШАХ,...,0,0,0,0,0,0,3,0,3,2025-03-18 16:40:02
